In [1]:
#Importing all the dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import requests
import mysql.connector

In [2]:
#fetch data from sales data from csv
sales_data=pd.read_csv("Sales data.csv")
sales_data.head()


,order_id,customer_id,product_id,quantity,price,order_date
0,2334,5,40,3,35.60,2022-06-21
1,6228,8,13,7,36.52,2023-03-08
2,7784,9,44,4,46.56,2023-04-22
3,6588,5,26,1,15.87,2022-10-23
4,5910,8,32,10,77.00,2022-10-05


In [3]:
#checking for duplicates
sales_data.duplicated().sum()

0

In [4]:
#checking for null values
sales_data.isnull().sum()

order_id       0
customer_id    0
product_id     0
quantity       0
price          0
order_date     0
dtype: int64

In [5]:
#fetching data from JSONPlaceholder API - /users for user_data
json_url="https://jsonplaceholder.typicode.com/users"
def get_user_data():
    response=requests.get(json_url)
    users=response.json()
    user_data=pd.json_normalize(users)
    return user_data

user_data=get_user_data()

In [6]:
user_data.head()

,id,name,username,email,phone,website,address.street,address.suite,address.city,address.zipcode,address.geo.lat,address.geo.lng,company.name,company.catchPhrase,company.bs
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496,Romaguera-Crona,Multi-layered client-server neural-net,harness real-time e-markets
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.9509,-34.4618,Deckow-Crist,Proactive didactic contingency,synergize scalable supply-chains
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.6102,-47.0653,Romaguera-Jacobson,Face to face bifurcated interface,e-enable strategic applications
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.4572,-164.2990,Robel-Corkery,Multi-tiered zero tolerance productivity,transition cutting-edge web services
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,Skiles Walks,Suite 351,Roscoeview,33263,-31.8129,62.5342,Keebler LLC,User-centric fault-tolerant solution,revolutionize end-to-end systems


In [7]:
#checking the info details
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   10 non-null     int64 
 1   name                 10 non-null     object
 2   username             10 non-null     object
 3   email                10 non-null     object
 4   phone                10 non-null     object
 5   website              10 non-null     object
 6   address.street       10 non-null     object
 7   address.suite        10 non-null     object
 8   address.city         10 non-null     object
 9   address.zipcode      10 non-null     object
 10  address.geo.lat      10 non-null     object
 11  address.geo.lng      10 non-null     object
 12  company.name         10 non-null     object
 13  company.catchPhrase  10 non-null     object
 14  company.bs           10 non-null     object
dtypes: int64(1), object(14)
memory usage: 1.3+ KB


In [8]:
#checking for duplicates
user_data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [9]:
#checking for null values
user_data.isnull().sum()

id                     0
name                   0
username               0
email                  0
phone                  0
website                0
address.street         0
address.suite          0
address.city           0
address.zipcode        0
address.geo.lat        0
address.geo.lng        0
company.name           0
company.catchPhrase    0
company.bs             0
dtype: int64

In [10]:
#Renaming columns and dropping unnecessary columns
rename_columns={"address.geo.lat":"lat","address.geo.lng":"lng","address.city":"city"}
drop_columns=["phone","website","address.street","address.suite","address.zipcode","company.name","company.catchPhrase","company.bs"]
user_data=user_data.rename(columns=rename_columns)
user_data=user_data.drop(columns=drop_columns)
user_data.head()

,id,name,username,email,city,lat,lng
0,1,Leanne Graham,Bret,Sincere@april.biz,Gwenborough,-37.3159,81.1496
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,Wisokyburgh,-43.9509,-34.4618
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,McKenziehaven,-68.6102,-47.0653
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,South Elvis,29.4572,-164.2990
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,Roscoeview,-31.8129,62.5342


In [25]:
#load the env variables
from dotenv import find_dotenv, load_dotenv
dotenv_pth=find_dotenv()
load_dotenv(dotenv_pth)

WEATHER_API_KEY=os.getenv("WEATHER_API_KEY")
#Necessary Weather columns
weather_columns = ['lat','lng','main_weather','description','temperature','feels_like','humidity','pressure']

#Function to fetch Weather data based on latitude and longitude
def get_weather_info(lat, lng):

    base_url =  "https://api.openweathermap.org/data/2.5/weather"#URL To fetch the data
    params = {
        "lat": lat,
        "lon": lng,
        "appid": WEATHER_API_KEY,
        "units": "metric"  
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} for (lat, lng): ({lat}, {lng})")
        return None

#Funtion to fetch weather information for all the entries in User Data
def create_weather_info(user_data):
 
    weather_data = pd.DataFrame(columns=weather_columns)  # Creating empty DataFrame inside the function

    for index, row in user_data.iterrows():
        latitude = row['lat']
        longitude = row['lng']
        weather_info = get_weather_info(latitude, longitude)
        print("got the data")

        if weather_info:
            # Extract relevant data
            data = {
                "address.geo.lat": latitude,
                "address.geo.lng": longitude,
                "main_weather": weather_info["weather"][0]["main"],
                "description": weather_info["weather"][0]["description"],
                "temperature": weather_info["main"]["temp"],
                "feels_like": weather_info["main"]["feels_like"],
                "humidity": weather_info["main"]["humidity"],
                "pressure": weather_info["main"]["pressure"]
            }

            # Create a single-row DataFrame from the dictionary and append it
            df = pd.DataFrame([data])
            weather_data=pd.concat([weather_data,df],axis=0)
            print("dataframe row appended")

    return weather_data

weather_data = create_weather_info(user_data.copy())

#check weather_data
weather_data.head()


KeyError: 'address.geo.lat'